# Keras to OpenVINO

In [13]:
%pip install -q "openvino>=2023.1.0" tensorflow keras-cv tqdm pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
import openvino as ov
import time
import warnings
import tarfile 
from pathlib import Path
import os

In [2]:
# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)
from notebook_utils import download_file

In [3]:
IMAGE_WIDTH = 150
IMAGE_HEIGHT = 150
DIRECTORY_NAME = "model"
KERAS_MODEL_BASE = DIRECTORY_NAME + "/keras_classification"
KERAS_MODEL_PATH = Path(KERAS_MODEL_BASE + "_model.h5")
KERAS_ZIP_PATH = Path(KERAS_MODEL_BASE + ".tar.gz")

### Loading Keras model

In [4]:
download_file("https://www.kaggle.com/api/v1/models/keras/resnetv2/keras/resnet50_v2_imagenet_classifier/2/download", filename=KERAS_ZIP_PATH.name, directory=KERAS_ZIP_PATH.parent)
zip_file = tarfile.open(KERAS_ZIP_PATH)
zip_file.extractall(KERAS_MODEL_BASE) 
zip_file.close()
os.remove(KERAS_ZIP_PATH)

model\keras_classification.tar.gz:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [5]:
KERAS_MODEL_PATH = Path(KERAS_MODEL_BASE + "/model.weights.h5")
KERAS_MODEL_CONFIG = Path(KERAS_MODEL_BASE + "/config.json")

In [6]:
import numpy as np
from keras.models import Sequential, model_from_json
import json

In [7]:
with open(KERAS_MODEL_CONFIG, 'r') as json_file:
    config = json.load(json_file)
keras_model = model_from_json(json.dumps(config))
keras_model.load_weights(KERAS_MODEL_PATH)

In [40]:
def predict(model,file):
    x = tf.keras.preprocessing.image.load_img(file, target_size=(IMAGE_WIDTH,IMAGE_HEIGHT))
    x = tf.keras.preprocessing.image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    # print("ARRAY: ", array)
    result = array[0]
    answer = np.argmax(result)
    # print("RESULT: ", result)
    if answer in range(236, 250):
        classify = 'dog'
    elif answer in range(280, 295):
        classify = 'cat'
    else:
        classify = 'else'
    print("ANSWER: ", answer, classify, Path(file).stem)

In [42]:
for i in range(1,4):
    name = f"dog{i}.jpg"
    result = predict(keras_model, name)
    name = f"cat{i}.jpg"
    result = predict(keras_model, name)
    name = f"non{i}.jpg"
    result = predict(keras_model, name)
    
predict(keras_model, "dog4.jpg")
predict(keras_model, "york.jpg")
predict(keras_model, "chihuahua.jpg")
predict(keras_model, "lio.jpg")
predict(keras_model, "wolf.jpg")
predict(keras_model, "weimar.jpg")
predict(keras_model, "buldog.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
ANSWER:  246 dog dog1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
ANSWER:  282 cat cat1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
ANSWER:  331 else non1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
ANSWER:  246 dog dog2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
ANSWER:  282 cat cat2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
ANSWER:  444 else non2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
ANSWER:  240 dog dog3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
ANSWER:  285 cat cat3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
ANSWER:  644 else non3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
ANSWER:  246 dog dog4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
ANSWER:  187 else york
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
ANSWER:  151 else chihuahua
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
ANSWER:  291 cat lio
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
ANSWER:  272 else wolf
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
ANSWER:  178 else weimar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
ANSWER:  180 else buldog


### Converting Keras model to TensorFlow Saved Model

In [9]:
TENSORFLOW_SAVEDMODEL_PATH = Path(DIRECTORY_NAME + "/tf_saved_model")
tf.saved_model.save(keras_model, TENSORFLOW_SAVEDMODEL_PATH)
print(f"Model TensorFlow saved at {TENSORFLOW_SAVEDMODEL_PATH}")

INFO:tensorflow:Assets written to: model\tf_saved_model\assets


INFO:tensorflow:Assets written to: model\tf_saved_model\assets


Model TensorFlow saved at model\tf_saved_model


### Converting TensorFlow to OpenVINO

In [20]:
OPENVINO_MODEL_PATH = Path(DIRECTORY_NAME + "/openvino_model" + "/model.xml")

In [23]:
print(OPENVINO_MODEL_PATH)

model\openvino_model\model.xml


In [31]:
input_shape = tuple(-1 if x is None else x for x in keras_model.input.shape)

In [33]:
TENSORFLOW_SAVEDMODEL_PATH

WindowsPath('model/tf_saved_model')

In [32]:
ov_model = ov.convert_model(TENSORFLOW_SAVEDMODEL_PATH, input = input_shape)
ov.save_model(ov_model, OPENVINO_MODEL_PATH)
print(f"Model OpenVINO saved at {OPENVINO_MODEL_PATH}")

OpConversionFailure: Check 'is_conversion_successful' failed at src\frontends\tensorflow\src\frontend.cpp:480:
FrontEnd API failed with OpConversionFailure:
[TensorFlow Frontend] Internal error, conversion is failed for VarHandleOp operation with a message:
Check 'result' failed at src\frontends\tensorflow\src\op\var_handle.cpp:99:
FrontEnd API failed with OpValidationFailureWhile validating node 'VarHandleOp':
[TensorFlow Frontend] Internal error: Cannot find requested variable.


